In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT

<h1> Stock pool </h1>

In [ ]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["AKRBP.OL", "EQNR.OL", "ODL.OL"]
ticker_df["Materials"] = ["NHY.OL", "NSKOG.OL", "RANA.OL"]
ticker_df["Food"] = ["LSG.OL", "MOWI.OL", "ORK.OL"]
ticker_df

In [ ]:
# Retrieve data from yf API
data = DatRet(ticker_df, "2022-02-19", "2025-02-18")
data.retrieve_data()

# Generate benchmark weights thorugh MPT using Sharpe ratio
benchmark = MPT(data.returns, 252,10)
benchmark.frequency_optimizing()

In [ ]:
flat_series_list = [series for row in data.returns for series in row]
df = pd.concat(flat_series_list,axis=1)

In [ ]:
df.tail(10)

In [50]:
# 

In [ ]:
daily_returns = []
for time in range(1,10,1):
    ind_day_ret = np.array([data.returns[i][j].iloc[-time].iloc[0] for i in  range(3) for j in range(3)]) -1
    daily_returns.append(ind_day_ret[::])
daily_returns = daily_returns[::-1]

daily_ben_weights = benchmark.frequency_weights[:-1]
daily_ben_weights = np.array([benchmark.frequency_weights[i][0] for i in range(len(benchmark.frequency_weights))])[:-1]
daily_exp_weights = np.repeat(1/9,9)


# rl_perf = [daily_ben_weights[time]@daily_returns[time] for time in range(9)]
rl_perf = [sum(daily_ben_weights[time]*daily_returns[time]) for time in range(9)]

# mpt_perf = [daily_exp_weights@daily_returns[time] for time in range(9)]
mpt_perf = [sum(daily_exp_weights*daily_returns[time]) for time in range(9)]

# excess_ret = [(rl_perf[time]-mpt_perf[time])/rl_perf[time] for time in range(9)]
excess_ret = [rl_perf[time] - mpt_perf[time] for time in range(9)]

plt.plot(excess_ret)

In [ ]:
def calculate_bhb(daily_ben_weights, daily_returns, daily_exp_weights):
    """
    Calculate Brinson, Hood, and Beebower (BHB) attribution model.

    Args:
        daily_ben_weights (list): List of 9 benchmark weights.
        daily_returns (list): List of 9 daily returns.
        daily_exp_weights (list): List of 9 experimental portfolio weights.

    Returns:
        tuple: Allocation effect, selection effect, interaction effect, and total excess return.
    """
    allocation_effect = 0
    selection_effect = 0
    interaction_effect = 0
    
    # Divide portfolio into 3 sectors (0:3, 3:6, 6:9)
    for i in range(3):
        wb = sum(daily_ben_weights[0][i*3:(i+1)*3])
        we = sum(daily_exp_weights[i*3:(i+1)*3])
        rb = sum(np.array(daily_ben_weights[0][i*3:(i+1)*3]) * np.array(daily_returns[0][i*3:(i+1)*3]))
        re = sum(np.array(daily_exp_weights[i*3:(i+1)*3]) * np.array(daily_returns[0][i*3:(i+1)*3]))
        
        allocation_effect += rb * (we - wb)  # Allocation Effect
        selection_effect += wb * (re - rb)  # Selection Effect
        interaction_effect += (we - wb) * (re - rb)  # Interaction Effect
    
    total_excess_return = allocation_effect + selection_effect + interaction_effect
    
    return allocation_effect, selection_effect, interaction_effect, total_excess_return


In [82]:
fox = calculate_bhb(daily_ben_weights,daily_returns, daily_exp_weights)

In [83]:
fox

(np.float64(-0.0005328568492038127),
 np.float64(-0.0011743012428612075),
 np.float64(0.00036851326751202317),
 np.float64(-0.001338644824552997))

In [84]:
sum(fox[:3])

np.float64(-0.001338644824552997)

In [79]:
wb1 = np.sum(daily_ben_weights[0][:3])
we1 = np.sum(daily_exp_weights[:3])
rb1 = np.dot(daily_ben_weights[0][:3], daily_returns[0][:3])
re1 = np.dot(daily_exp_weights[:3], daily_returns[0][:3])

wb2 = np.sum(daily_ben_weights[0][3:6])
we2 = np.sum(daily_exp_weights[3:6])
rb2 = np.dot(daily_ben_weights[0][:3], daily_returns[0][3:6])
re2 = np.dot(daily_exp_weights[:3], daily_returns[0][3:6])


wb3 = np.sum(daily_ben_weights[0][6:9])
we3 = np.sum(daily_exp_weights[6:9])
rb3 = np.dot(daily_ben_weights[0][:3], daily_returns[0][6:9])
re3 = np.dot(daily_exp_weights[:3], daily_returns[0][6:9])

# Allocations:
ae1 = rb1*(we1-wb1)
ae2 = rb2*(we2-wb2)
ae3 = rb3*(we3-wb3)
allocation = ae1+ae2+ae3

# Selections:
se1 = wb1*(re1-rb1)
se2 = wb2*(re2-rb2)
se3 = wb3*(re3-rb3)
selection = se1+se2+se3

# Interaction:
ie1 = (we1-wb1)*(re1-rb1)
ie2 = (we2-wb2)*(re2-rb2)
ie3 = (we3-wb3)*(re3-rb3)
interaction = ie1 + ie2+ ie3

print(allocation*100)
print(selection*100)
print(interaction*100)
delta=allocation+selection+interaction
print(delta)

-0.006934288266357137
0.0015404291477781572
-0.009500069902821807
-0.00014893929021400786


In [80]:
fox

(np.float64(-0.0005328568492038127),
 np.float64(-0.0011743012428612075),
 np.float64(0.00036851326751202317),
 np.float64(-0.001338644824552997))

In [68]:
print(delta-1)
print(excess_ret[0]+1)

-0.014893929021400742
1.0024173639260476


In [61]:
# class BHBAnalyzer():
    
#     def __init__(self, 
#                  benchmark_data=None,
#                  experiment_data=None,
#                  raw_data=None
#                  ):
#         """
#         Args:
#             benchmark_data: NxM list from MPT
#             experiment_data: NxM list from RL    
#         """
#         self.benchmark_data =  benchmark_data
#         self.experiment_data = experiment_data
#         self.raw_data = raw_data

#     def frequency_analyze():
#         pass